# Imports

In [2]:
import statsmodels.api as sm
import pandas as pd
import pyodbc
from functools import reduce

# Data retrieval
public and private 4 year schools in states, financials

In [15]:
#Database folder and tables...and year
Public_file_table_dict = {
'IPEDS201920': ['F1819_F1A'],
'IPEDS201819': ['F1718_F1A'],
'IPEDS201718': ['F1617_F1A'],
'IPEDS201617': ['F1516_F1A'],
'IPEDS201516': ['F1415_F1A'],
'IPEDS201415': ['F1314_F1A']
}

In [55]:

# pulling 6 years of financial data
def access_to_csv(dict, title):
    '''Function for pulling data from ACCESS databases. 
    Then, after quering for select columns and renaming the table keys, merge the dataframes together
    Finally, saving the merge to a csv to pull at a later time.
    '''
    frames=[]
    for j, k  in dict.items():
        conn_str = (r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
            f'DBQ=/Users/andy.schmeck.OFFICE/Documents/flatiron/phase_5/IPEDS/{j}.accdb;')
        conn = pyodbc.connect(conn_str) 
        #query first table and add year column
        df1 = pd.read_sql(f'select * from {k[0]}', conn)
        #query second table
        df2 = pd.read_sql(f'select UNITID, GBA6RTBK from {k[1]}', conn)
        #query third table
        df3 = pd.read_sql(f'select unitid, F1CORREV, F1TUFEPC,F1STAPPC,F1LCAPPC,\
            F1GVGCPC,F1PGGCPC,F1INVRPC,F1OTRVPC,F1TUFEFT,F1STAPFT,F1LCAPFT,\
            F1GVGCFT,F1PGGCFT,F1INVRFT,F1OTRVFT,F1COREXP,F1INSTPC,F1RSRCPC,\
            F1PBSVPC,F1ACSPPC,F1STSVPC,F1INSUPC,F1OTEXPC,F1INSTFT,F1RSRCFT,\
            F1PBSVFT,F1ACSPFT,F1STSVFT,F1INSUFT,F1OTEXFT,F1SAFBPC,F1SALRPC,\
            F1ENDMFT,F1EQUITR from {k[2]}', conn)
        #rename key
        df3.rename(columns={'unitid': "UNITID"}, inplace=True)
        #apphend the dataframes
        tables = [df1,df2,df3]
        table= reduce(lambda x, y: pd.merge(x, y, how = 'outer', on = 'UNITID'), tables)
        frames.append(table)
        #merge the frame on key
    df= reduce(lambda x, y: pd.merge(x, y, how = 'outer', on = 'UNITID'), frames)
    df.to_csv(f'{title}.csv')

In [56]:
#call the function
access_to_csv(Public_file_table_dict, 'Public_Financials')

In [57]:
df_public = pd.read_csv("data/Public_Financials.csv").iloc[:, 1:]
df_public.set_index(['UNITID'], inplace=True)
df_public

,F1A01_x,F1A31_x,F1A04_x,F1A05_x,F1A06_x,F1A19_x,F1A07_x,F1A08_x,F1A09_x,F1A10_x,...,F1RSRCFT_y.2,F1PBSVFT_y.2,F1ACSPFT_y.2,F1STSVFT_y.2,F1INSUFT_y.2,F1OTEXFT_y.2,F1SAFBPC_y.2,F1SALRPC_y.2,F1ENDMFT_y.2,F1EQUITR_y.2
UNITID,,,,,,,,,,,,,,,,,,,,,
100654,1.049150e+08,1.369788e+08,2.605815e+07,1.630370e+08,2.679520e+08,17535476.0,3691441.0,81181209.0,84872650.0,8.105261e+07,...,2277.0,3888.0,1429.0,3758.0,6019.0,3139.0,50.0,39.0,NaN,44.0
100663,1.817827e+09,1.718187e+09,1.613978e+09,3.332165e+09,5.149992e+09,313741542.0,35380749.0,444691190.0,480071939.0,1.127091e+09,...,15851.0,8067.0,10986.0,2153.0,8594.0,1286.0,56.0,43.0,25985.0,66.0
100706,1.150075e+08,2.950597e+08,1.119759e+08,4.070356e+08,5.220431e+08,35809718.0,5308149.0,85200348.0,90508497.0,1.477739e+08,...,14221.0,1045.0,2097.0,2811.0,3644.0,185.0,71.0,54.0,12457.0,64.0
100724,5.677111e+07,2.635835e+08,1.262004e+06,2.648455e+08,3.216166e+08,26123806.0,10399795.0,28011480.0,38411275.0,2.039958e+08,...,455.0,1417.0,2238.0,3227.0,6477.0,3582.0,46.0,36.0,14885.0,28.0
100733,3.116550e+06,1.240667e+07,1.324194e+07,2.564861e+07,2.876516e+07,5284335.0,0.0,3566251.0,3566251.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,82.0,62.0,NaN,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_private = pd.read_csv("data/Private_Financials.csv").iloc[:, 1:]
df_private.set_index(['UNITID'], inplace=True)
df_private


,F2A01_x,F2A19_x,F2A20_x,F2A02_x,F2A03_x,F2A03A_x,F2A04_x,F2A05_x,F2A05A_x,F2A05B_x,...,F1PBSVFT_y.2,F1ACSPFT_y.2,F1STSVFT_y.2,F1INSUFT_y.2,F1OTEXFT_y.2,F1SAFBPC_y.2,F1SALRPC_y.2,F1ENDMFT_y.2,F1EQUITR_y.2,year_y.8
UNITID,,,,,,,,,,,,,,,,,,,,,
100690,9384788.0,2634979.0,0.0,14438530.0,2487355.0,1296697.0,11242127.0,709048.0,174805.0,534243.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
100937,50743827.0,89400865.0,0.0,159285516.0,47756472.0,34000110.0,53054909.0,58474135.0,7669338.0,50804797.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
101189,20868071.0,53530199.0,0.0,89941892.0,42026935.0,33512123.0,29345184.0,18569773.0,13162812.0,5406961.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
101365,0.0,3242987.0,0.0,9435073.0,3717479.0,0.0,5717594.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
101435,50818280.0,29298003.0,0.0,86309501.0,26153792.0,22307594.0,9068001.0,51087708.0,47866409.0,3221299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
481173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0
481827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0


# Data Cleaning
Nans

# Feature Creation
rate of change for graduation rates

# Target and Split
find 7-year average graduation rate; bin target "above average/below average"

# Modeling
Pipeline, Dummy Classifier, Decision Tree

### Pipeline

### Dummy

### Decision Tree

In [39]:
#Import all the needed modules
import numpy as np
import pandas as pd
from functools import reduce
import statsmodels.api as sm

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

In [54]:
df = pd.read_csv("data/Gr_Lakes_public_financials.csv").iloc[:, 1:]
df.set_index(['year','UNITID'], inplace=True)

In [55]:
# df = df.corr().abs().stack().reset_index().sort_values(0, ascending=False)
# df = X.corr().abs().stack().reset_index().sort_values(0, ascending=False)
# df['pairs'] = list(zip(df.level_0, df.level_1))
# df.set_index(['pairs'], inplace = True)
# df.drop(columns=['level_1', 'level_0'], inplace = True)
# df.columns = ['cc']
# df.drop_duplicates(inplace=True)
# df[(df.cc>.75) & (df.cc <1)]


In [56]:
#Create X and y dataframes and train-test split them
y = df['GBA6RTBK']
X = df.drop(columns = ['GBA6RTBK'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [57]:
#Concatenate to one dataframe, check for nan's
df = pd.concat([X_train, y_train], axis=1)
df.isna().sum().sort_values(ascending=False)

F1M07     726
F1M08     726
F1M06     726
F1M05     726
F1A19     522
         ... 
F1C012     12
F1C021     12
F1C022     12
F1C031     12
F1E06      12
Length: 211, dtype: int64

In [58]:
df.shape

(780, 211)

In [59]:
#drop columns with 40% missing values
res2 = df.columns[df.isnull().sum() > 311]
df.drop(res2, inplace=True, axis=1)
df.dropna(inplace=True)

In [60]:
#split back
y_train = df['GBA6RTBK']
X_train = df.drop(columns = ['GBA6RTBK'], axis = 1)



In [61]:
#bin target to binary
y_train.loc[y_train > .604] = 1
y_train.loc[y_train < .604] = 0

In [77]:
X_train = X_train[['F1B19','F1STSVPC','F1TUFEFT','F1E09','F1A14','F1OTEXFT']]

In [78]:
#Set up pipeline for scaling continuous variables
continuous_pipeline = Pipeline(steps=[
    ('ss', StandardScaler())
])

In [79]:
trans = ColumnTransformer(transformers=[
    ('continuous', continuous_pipeline, X_train.columns),
])

In [80]:
model_one = Pipeline(steps=[
    ('trans', trans),
    ('simple_dt', DecisionTreeClassifier(max_depth = 5, random_state = 42))
])
#Fit model on all the data
model_one.fit(X_train, y_train)
#Grab predictions and print precision
y_pred = model_one.predict(X_train)
print("Training Score:" + str(accuracy_score(y_train, y_pred)))
#Run a cross validation to test for overfitting
scores = np.mean(cross_val_score(model_one, X_train, y_train, cv=5, scoring = 'accuracy'))
print("Validation Score:" + str(scores))

Training Score:1.0
Validation Score:0.9805750350631136


In [81]:
for name, importance in zip(X_train.columns, model_one['simple_dt'].feature_importances_):
    print(name, importance)

F1B19 0.35259178673297703
F1STSVPC 0.03081664371763838
F1TUFEFT 0.26798418972332017
F1E09 0.1955551086082057
F1A14 0.0909712843581109
F1OTEXFT 0.06208098685974785


In [82]:
X_train.corr()

,F1B19,F1STSVPC,F1TUFEFT,F1E09,F1A14,F1OTEXFT
F1B19,1.000000,-0.518087,0.648069,0.597588,0.854113,0.271116
F1STSVPC,-0.518087,1.000000,-0.583611,-0.366469,-0.462333,-0.126459
F1TUFEFT,0.648069,-0.583611,1.000000,0.641629,0.690133,0.167303
F1E09,0.597588,-0.366469,0.641629,1.000000,0.708646,0.107830
F1A14,0.854113,-0.462333,0.690133,0.708646,1.000000,0.274963
F1OTEXFT,0.271116,-0.126459,0.167303,0.107830,0.274963,1.000000


# Phase 3 Code

In [10]:
#pipeline
pipeline = Pipeline(steps=[
    ('si', SimpleImputer(missing_values=np.nan))
])

# column transformer
trans = ColumnTransformer(transformers=[
    ('pipeline', pipeline, X_train.columns)
])

#Pipeline for running the model
dummy = Pipeline(steps=[
    ('trans', trans),
    ('dummy', DummyClassifier(random_state = 42, strategy = '' ))
])

SyntaxError: invalid syntax (<ipython-input-10-3fc4cefdca38>, line 14)

In [ ]:


#Bin and fill in nulls in installer
inst_five = X_train.installer.value_counts(sort = True, ascending = False)[:5]
inst_list = list(inst_five.index)
for idx, value in enumerate(inst_list):
    inst_list[idx] = value.lower()
X_train['installer'] = X_train['installer'].apply(install_bin)
X_test['installer'] = X_test['installer'].apply(install_bin)
      
#Bin and fill in nulls in scheme_management
scheme_eight = X_train.scheme_management.value_counts(sort = True, ascending = False)[:9]
scheme_list = list(scheme_eight.index)
for idx, value in enumerate(scheme_list):
    scheme_list[idx] = value.lower() 
X_train['scheme_management'] = X_train['scheme_management'].apply(scheme_bin)
X_test['scheme_management'] = X_test['scheme_management'].apply(install_bin)

#Create categorical and continuous feature split
X_train_cat = X_train.select_dtypes('object')
X_train_cont = X_train.select_dtypes(['float64', 'int64'])

#Set up pipeline for scaling continuous variables
continuous_pipeline = Pipeline(steps=[
    ('ss', StandardScaler())
])

#Set up pipeline for encoding categorical variables
categorical_pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first'))
])

#Bind the scaling and encoding process together
trans = ColumnTransformer(transformers=[
    ('continuous', continuous_pipeline, X_train_cont.columns),
    ('categorical', categorical_pipeline, X_train_cat.columns)
])

#Pipeline for running the model
dummy = Pipeline(steps=[
    ('trans', trans),
    ('dummy', DummyClassifier(random_state = 42, strategy = 'most_frequent'))
])

#Fitting and checking the score
dummy.fit(X_train, y_train)
dummy.score(X_train, y_train)

In [ ]:
#Pipeline for decision tree
model_one = Pipeline(steps=[
    ('trans', trans),
    ('simple_dt', DecisionTreeClassifier(max_depth = 5, random_state = 42))
])

#Fit model on all the data
model_one.fit(X_train, y_train)
#Grab predictions and print precision
y_pred = model_one.predict(X_train)
print("Training Score:" + str(precision_score(y_train, y_pred)))
#Run a cross validation to test for overfitting
scores = np.mean(cross_val_score(model_one, X_train, y_train, cv=5, scoring = 'precision'))
print("Validation Score:" + str(scores))

In [ ]:
for name, importance in zip(X_train.columns, model_one['simple_dt'].feature_importances_):
    print(name, importance)

In [ ]:
#Create new features dataframe based on results above
X = df_trim[['amount_tsh', 'permit', 'installer', 'extraction_type_class']]

#Split the data again
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

#Bin installer again
inst_five = X_train.installer.value_counts(sort = True, ascending = False)[:5]
inst_list = list(inst_five.index)
for idx, value in enumerate(inst_list):
    inst_list[idx] = value.lower()
X_train['installer'] = X_train['installer'].apply(install_bin)
X_test['installer'] = X_test['installer'].apply(install_bin)

#Separate out which columns are categorical or continuous
X_train_cat = ['permit', 'installer', 'extraction_type_class']
X_train_cont = ['amount_tsh']

#Adjust transformer to account for change in assigning X_train_cont
trans = ColumnTransformer(transformers=[
    ('continuous', continuous_pipeline, X_train_cont),
    ('categorical', categorical_pipeline, X_train_cat)
])

#Pipeline for logistic regression
logreg = Pipeline(steps=[
    ('trans', trans),
    ('logr', LogisticRegression(random_state = 42))
])

#Fit the model
logreg.fit(X_train, y_train)
#Print precision or training and validation sets
y_pred = logreg.predict(X_train)
print("Training Score:" + str(precision_score(y_train, y_pred)))
scores = np.mean(cross_val_score(logreg, X_train, y_train, cv=5, scoring = 'precision'))
print("Validation Score:" + str(scores))

In [3]:
#check assumptions

In [4]:
#Grab probabilities and calculate log odds
pred = logreg.predict_proba(X_train)[:, 0]
log_odds = np.log(pred / (1 - pred))
#Plot log odds versus continuous variable to check for linearity
plt.scatter(x = X_train['amount_tsh'].values, y = log_odds)
plt.title("Logistic Regression Assumption Test")
plt.xlabel("amount_tsh")
plt.ticklabel_format(axis='x', style='sci', scilimits=(0,0))
plt.ylabel("Log-odds")
plt.show();

NameError: name 'logreg' is not defined

In [ ]:
#Build pipeline for random forest
ensemble = Pipeline(steps=[
    ('trans', trans),
    ('rfc', RandomForestClassifier(random_state = 42))
])

#Fit the model
ensemble.fit(X_train, y_train)
#Print out precision for training and validation
y_pred = ensemble.predict(X_train)
print("Training Score:" + str(precision_score(y_train, y_pred)))
scores = np.mean(cross_val_score(ensemble, X_train, y_train, cv=5))
print("Validation Score:" + str(scores))

In [ ]:
#Avoid a long runtime, the code is included but hashed out
"""
#Create parameters to test
params = {
    'rfc__criterion': ['gini', 'entropy'],
    'rfc__n_estimators': [100, 300, 500],
    'rfc__min_samples_split': [2, 5, 10]
}

#Fit gridsearch on model and prints out the best parameters
search = GridSearchCV(ensemble, param_grid = params, scoring = 'precision')
search.fit(X_train, y_train)
search.best_params_
""";

In [ ]:
ensemble_tuned = Pipeline(steps=[
    ('trans', trans),
    ('rfc', RandomForestClassifier(criterion = 'entropy', min_samples_split = 5, 
                                   n_estimators = 300, random_state = 42))
])

#Fit and print precision for tuned model
ensemble_tuned.fit(X_train, y_train)
y_pred = ensemble_tuned.predict(X_train)
print("Training Score:" + str(precision_score(y_train, y_pred)))
scores = np.mean(cross_val_score(ensemble_tuned, X_train, y_train, cv=5))
print("Validation Score:" + str(scores))

# evaluation

In [ ]:
print("Training Accuracy:" + str(logreg.score(X_train, y_train)))
scores = np.mean(cross_val_score(logreg, X_train, y_train, cv=5))
print("Validation Accuracy:" + str(scores))

In [ ]:
#Generate precision score for test set
test_pred = logreg.predict(X_test)
print("Test Score:" + str(precision_score(y_test, test_pred)))
print("Tets Accuracy:" + str(logreg.score(X_test, y_test)))